# Adsorption Models - Exercise

## Exercise 1: Breakthrough curve in a Column

Let us start working towards the task given by our colleagues. Try to set up your column dimensions to match the data from the lab book.

Two parameters are missing (we will fit them later). Take these placeholder values for now:

```
total_porosity = 0.4
axial_dispersion = 1e-6
```


We will stay in a simplified binding model for now by using the `Langmuir` binding model with two components with $k_{eq, i} = [2, 1]$, and $q_{max, i} = [1, 1]$.

Load the column with $c_{feed} = [1.0, 1.0]~mmol \cdot L^{-1} $ and a flow rate of $1 ~ml \cdot min^{-1}$ for $100~s$.

Try experimenting with:
- The time resolution.
- The binding strenth.
- The number of components.
- Modify how many components bind.
- Compare kinetic binding with rapid equilibrium.

In [ ]:
from CADETProcess.processModel import ComponentSystem

component_system = ComponentSystem(['A', 'B'])

from CADETProcess.processModel import Langmuir

binding_model = Langmuir(component_system, name='linear')
binding_model.is_kinetic = False
binding_model.adsorption_rate = [2, 1]
binding_model.desorption_rate = [1, 1]
binding_model.capacity = [1, 1]

from CADETProcess.processModel import Inlet, Outlet, LumpedRateModelWithoutPores

column = LumpedRateModelWithoutPores(component_system, name="reactor")
column.binding_model = binding_model

column.length = 0.014
column.total_porosity = 0.4
column.diameter = 0.01
column.axial_dispersion = 1e-7

inlet = Inlet(component_system, name='inlet')
inlet.c = [1.0, 1.0]
inlet.flow_rate = 1.67e-8

outlet = Outlet(component_system, name= 'outlet')


from CADETProcess.processModel import FlowSheet

flow_sheet = FlowSheet(component_system)

flow_sheet.add_unit(column)
flow_sheet.add_unit(inlet)
flow_sheet.add_unit(outlet)

flow_sheet.add_connection(inlet,column)
flow_sheet.add_connection(column,outlet)

from CADETProcess.processModel import Process

process = Process(flow_sheet, 'process')
process.cycle_time = 100

from CADETProcess.simulator import Cadet
simulator = Cadet()
simulator.time_resolution = 1

sim_results = simulator.run(process)
_ = sim_results.solution.reactor.outlet.plot()

## Exercise 2: SMA Model in Column

Now get closer to the experiment performed by your colleague by using the `StericMassAction` adsorption model. Note, that the `StericMassAction` requires the first component to be a binding salt component.

For the binding model use these placeholder parameters:
```
is_kinetic = True
adsorption_rate = [0, 1e-5, 1e-1, 1e-3]
desorption_rate = [0, 1, 1, 1]
characteristic_charge = [0, 5, 1, 7]
steric_factor = [0, 1, 1, 1]
```
we will fit their true values later. You can find the capacity in the lab book.

Use a feed concentration of $c_{feed} = [50.0, 1.0, 1.0, 1.0]~mol \cdot m^{-3} $.

Try experimenting with:
- The binding strenth.
- The number of components.
- Modify how many components bind.
- Compare kinetic binding with rapid equilibrium.

In [ ]:
from CADETProcess.processModel import ComponentSystem

component_system = ComponentSystem(["Salt", 'A', 'B', "C"])

from CADETProcess.processModel import StericMassAction

binding_model = StericMassAction(component_system, name='langmuir')
binding_model.is_kinetic = True
binding_model.adsorption_rate = [0, 1e-5, 1e-1, 1e-3]
binding_model.desorption_rate = [0, 1, 1, 1]
binding_model.characteristic_charge = [0, 5, 1, 7]
binding_model.steric_factor = [0, 1, 1, 1]
binding_model.capacity = 1200
print(binding_model.required_parameters)

from CADETProcess.processModel import Inlet, Outlet, LumpedRateModelWithoutPores

column = LumpedRateModelWithoutPores(component_system, name='column')
column.length = 0.014
column.total_porosity = 0.4
column.diameter = 0.01
column.axial_dispersion = 1e-7

column.binding_model = binding_model

column.c=[50, 0, 0, 0]
column.q=[50, 0, 0, 0]

inlet = Inlet(component_system, name='inlet')
inlet.c = [50., 1, 1, 1]
inlet.flow_rate = 1.67e-8

outlet = Outlet(component_system, name='outlet')

from CADETProcess.processModel import FlowSheet

flow_sheet = FlowSheet(component_system)

flow_sheet.add_unit(column)
flow_sheet.add_unit(inlet)
flow_sheet.add_unit(outlet)

flow_sheet.add_connection(inlet,column)
flow_sheet.add_connection(column,outlet)

from CADETProcess.processModel import Process

process = Process(flow_sheet, 'process')
process.cycle_time = 1000

from CADETProcess.simulator import Cadet
simulator = Cadet()
sim_results = simulator.run(process)

In [ ]:
from CADETProcess.plotting import SecondaryAxis

sec = SecondaryAxis()
sec.components = ['Salt']
sec.y_label = '$c_{salt}$'

_ = sim_results.solution.outlet.outlet.plot(secondary_axis=sec)

## Bonus Exercise

There are many other models implemented in `CADET`.
Some binding models can account for multiple binding states, others allow for mobile phase modulators.
Moreover, binding models can use external functions for modifying the values of the parameters during the simulation.
This could be used to model other effects like temperature that have an influence on the binding strength.

Try implementing some of the more advanced isotherms and features.